In [2]:
import ml_utils as mu
import numpy as np
from xgboost.sklearn import XGBClassifier
import xgboost as xgb

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.feature_selection import SelectKBest, f_classif


# get the training data
data = mu.get_training_data()
X, y = mu.split_x_and_y(data)
X = add_fa_fw_ratio(X)

# get the holdout and outside validation data
Xh, yh = mu.split_x_and_y(mu.get_holdout_data())
Xh = add_fa_fw_ratio(Xh)
from IPython.display import display
Xh, yh = mu.resample_to_equal_class_sizes(Xh, yh)

Xv, yv = mu.split_x_and_y(mu.get_validation_data())
# Xv = add_fa_fw_ratio(Xv)
Xv, yv = mu.resample_to_equal_class_sizes(Xv, yv)


std = StandardScaler().fit(X)
X = std.transform(X)
Xh = std.transform(Xh)
Xv = std.transform(Xv)

pca = PCA(n_components=10).fit(X)
lda = LinearDiscriminantAnalysis().fit(X, y)
kbest = SelectKBest(score_func=f_classif, k=10).fit(X,y)

X  = np.hstack([kbest.transform(X),  pca.transform(X),  lda.transform(X)])
Xh = np.hstack([kbest.transform(Xh), pca.transform(Xh), lda.transform(Xh)])
Xv = np.hstack([kbest.transform(Xv), pca.transform(Xv), lda.transform(Xv)])

X, y = mu.resample_to_equal_class_sizes(X, y)

Maximum class size is 80
Class 0 size is 48. Resampling with replacement to 80
Class 1 size has max class size (80).
Class 2 size is 11. Resampling with replacement to 80
Class 3 size is 11. Resampling with replacement to 80
Maximum class size is 32
Class 0 size is 6. Resampling with replacement to 32
Class 1 size has max class size (32).
Class 2 size is 4. Resampling with replacement to 32
Class 3 size is 8. Resampling with replacement to 32
Maximum class size is 319
Class 0 size is 192. Resampling with replacement to 319
Class 1 size has max class size (319).
Class 2 size is 41. Resampling with replacement to 319
Class 3 size is 44. Resampling with replacement to 319


In [3]:
print(X.shape)
print(Xh.shape)
print(Xv.shape)

(1276, 23)
(320, 23)
(128, 23)


In [5]:
clf = XGBClassifier( learning_rate =0.01, n_estimators=2000, nthread=-1)

#eval_set = [(Xh, yh)]
clf.fit(X,y)
clf.set_params(**{'subsample': 1.0, 'min_child_weight': 1, 'max_depth': 5, 'gamma': 0.5, 'colsample_bytree': 0.6})


print('Model Accuracy: %f' %clf.score(X,y))
print('Holdout Accuracy: %f' %clf.score(Xh,yh))
print('Validation Accuracy: %f' %clf.score(Xv,yv))


Model Accuracy: 1.000000
Holdout Accuracy: 0.787500
Validation Accuracy: 0.593750


In [ ]:
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(estimator=clf, param_grid=params, scoring='accuracy', n_jobs=4, cv=skf.split(X,y), verbose=3 )
grid.fit(X, y)
print('\n All results:')
print(grid.cv_results_)
print('\n Best estimator:')
print(grid.best_estimator_)
print('\n Best score:')
print(grid.best_score_ * 2 - 1)
print('\n Best parameters:')
print(grid.best_params_)
results = pd.DataFrame(grid.cv_results_)
results.to_csv('xgb-grid-search-results-01.csv', index=False)

y_test = grid.best_estimator_.predict_proba(test)
results_df = pd.DataFrame(data={'id':test_df['id'], 'target':y_test[:,1]})
results_df.to_csv('submission-grid-search-xgb-porto-01.csv', index=False)

Fitting 3 folds for each of 405 candidates, totalling 1215 fits


[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:   32.7s
[Parallel(n_jobs=4)]: Done 120 tasks      | elapsed:  2.6min
[Parallel(n_jobs=4)]: Done 280 tasks      | elapsed:  6.2min
